In [1]:
!pip install torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 15.3 MB/s eta 0:00:00


In [2]:
!pip install torchinfo

In [3]:
import matplotlib.pyplot as plt # for visualizing
import os
import torch
from torch import nn
import torchinfo
import torchvision
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import ToTensor
import math
import torch.nn as nn
from collections import OrderedDict

import torch.backends.cudnn as cudnn
import argparse


import numpy as np

print(torch.__version__)
print(torchvision.__version__)

2.8.0+cu126
0.23.0+cu126


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

드라이브 연결

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
origin_dir = '/content/drive/MyDrive/Colab_Notebooks/URP/VGG16_CIFAR100_pruning'
data_dir = origin_dir + '/data'
job_dir = origin_dir + '/pruning_job'

In [7]:
print(job_dir)

/content/drive/MyDrive/Colab_Notebooks/URP/VGG16_CIFAR100_pruning/pruning_job


데이터셋 불러오기

In [8]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),   # 데이터 증강: 랜덤 크롭
    transforms.RandomHorizontalFlip(),      # 좌우 반전
    transforms.AutoAugment(policy=transforms.AutoAugmentPolicy.CIFAR10),
    #데이터 증강 기법을 자동으로 탐색 후 적용
    transforms.ToTensor(),                  # Tensor 변환
    transforms.Normalize((0.5071, 0.4867, 0.4408),
                         (0.2675, 0.2565, 0.2761))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408),
                         (0.2675, 0.2565, 0.2761)),
])

trainset = torchvision.datasets.CIFAR100(
    root = data_dir, train=True, download=True, transform=transform_train
)

testset = torchvision.datasets.CIFAR100(
    root= data_dir, train=False, download=True, transform=transform_test
)


In [9]:
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=2
)

testloader = torch.utils.data.DataLoader(
    testset, batch_size=128, shuffle=False, num_workers=2
)

trainloader,testloader

(<torch.utils.data.dataloader.DataLoader at 0x7c529bc2bd40>,
 <torch.utils.data.dataloader.DataLoader at 0x7c52a5f36930>)

Logger 정의

In [10]:
import logging

def get_logger(file_path):
    """ Make python logger """
    # [!] Since tensorboardX use default logger (e.g. logging.info()),
    # we should use custom logger
    logger = logging.getLogger("kd")

    log_format = "%(asctime)s | %(message)s"
    formatter = logging.Formatter(log_format, datefmt="%m/%d %I:%M:%S %p")

    file_handler = logging.FileHandler(file_path)
    file_handler.setFormatter(formatter)

    stream_handler = logging.StreamHandler()
    stream_handler.setFormatter(formatter)

    logger.addHandler(file_handler)
    logger.addHandler(stream_handler)
    logger.setLevel(logging.INFO)

    return logger

In [11]:
import sys
import time
import shutil

TOTAL_BAR_LENGTH = 65.
last_time = time.time()
begin_time = last_time
term_width = shutil.get_terminal_size().columns
def format_time(seconds):
    days = int(seconds / 3600/24)
    seconds = seconds - days*3600*24
    hours = int(seconds / 3600)
    seconds = seconds - hours*3600
    minutes = int(seconds / 60)
    seconds = seconds - minutes*60
    secondsf = int(seconds)
    seconds = seconds - secondsf
    millis = int(seconds*1000)

    f = ''
    i = 1
    if days > 0:
        f += str(days) + 'D'
        i += 1
    if hours > 0 and i <= 2:
        f += str(hours) + 'h'
        i += 1
    if minutes > 0 and i <= 2:
        f += str(minutes) + 'm'
        i += 1
    if secondsf > 0 and i <= 2:
        f += str(secondsf) + 's'
        i += 1
    if millis > 0 and i <= 2:
        f += str(millis) + 'ms'
        i += 1
    if f == '':
        f = '0ms'
    return f

In [12]:
def progress_bar(current, total, msg=None):
    global last_time, begin_time
    if current == 0:
        begin_time = time.time()  # Reset for new bar.

    cur_len = int(TOTAL_BAR_LENGTH*current/total)
    rest_len = int(TOTAL_BAR_LENGTH - cur_len) - 1

    sys.stdout.write(' [')
    for i in range(cur_len):
        sys.stdout.write('=')
    sys.stdout.write('>')
    for i in range(rest_len):
        sys.stdout.write('.')
    sys.stdout.write(']')

    cur_time = time.time()
    step_time = cur_time - last_time
    last_time = cur_time
    tot_time = cur_time - begin_time

    L = []
    L.append('  Step: %s' % format_time(step_time))
    L.append(' | Tot: %s' % format_time(tot_time))
    if msg:
        L.append(' | ' + msg)

    msg = ''.join(L)
    sys.stdout.write(msg)
    for i in range(term_width-int(TOTAL_BAR_LENGTH)-len(msg)-3):
        sys.stdout.write(' ')

    # Go back to the center of the bar.
    for i in range(term_width-int(TOTAL_BAR_LENGTH/2)+2):
        sys.stdout.write('\b')
    sys.stdout.write(' %d/%d ' % (current+1, total))

    if current < total-1:
        sys.stdout.write('\r')
    else:
        sys.stdout.write('\n')
    sys.stdout.flush()

In [13]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0.0
        self.avg = 0.0
        self.sum = 0.0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [14]:
def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [15]:
logger = logging.getLogger("kd")
logger.handlers.clear()   # 모든 핸들러 제거

print_logger = get_logger(job_dir + '/logger.log')
print_logger.info('==> Preparing data..')
print(print_logger.handlers)

10/03 11:26:05 AM | ==> Preparing data..
INFO:kd:==> Preparing data..


[<FileHandler /content/drive/MyDrive/Colab_Notebooks/URP/VGG16_CIFAR100_pruning/pruning_job/logger.log (NOTSET)>, <StreamHandler stderr (NOTSET)>]


모델 생성

In [16]:
import math
import torch.nn as nn
from collections import OrderedDict

norm_mean, norm_var = 0.0, 1.0

defaultcfg = [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 512]
relucfg = [2, 6, 9, 13, 16, 19, 23, 26, 29, 33, 36, 39]
convcfg = [0, 3, 7, 10, 14, 17, 20, 24, 27, 30, 34, 37]


class VGG(nn.Module):
    def __init__(self, num_classes=100, init_weights=True, cfg=None):
        super(VGG, self).__init__()
        self.features = nn.Sequential()

        if cfg is None:
            cfg = defaultcfg

        self.relucfg = relucfg
        self.covcfg = convcfg
        self.features = self.make_layers(cfg[:-1], True)
        self.classifier = nn.Sequential(OrderedDict([
            ('linear1', nn.Linear(cfg[-2], cfg[-1])),
            ('norm1', nn.BatchNorm1d(cfg[-1])),
            ('relu1', nn.ReLU(inplace=True)),
            ('linear2', nn.Linear(cfg[-1], num_classes)),
        ]))

        if init_weights:            #weight 초기화
            self._initialize_weights()

    def make_layers(self, cfg, batch_norm=True):
        layers = nn.Sequential()
        in_channels = 3
        cnt = 0
        for i, v in enumerate(cfg):
            if v == 'M':
                layers.add_module('pool%d' % i, nn.MaxPool2d(kernel_size=2, stride=2))
            else:
                conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)

                layers.add_module('conv%d' % i, conv2d)
                layers.add_module('norm%d' % i, nn.BatchNorm2d(v))
                layers.add_module('relu%d' % i, nn.ReLU(inplace=True))
                in_channels = v
        return layers

    def forward(self, x):
        x = self.features(x)

        x = nn.AvgPool2d(2)(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(0.5)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.weight.data.normal_(0, 0.01)
                m.bias.data.zero_()

In [48]:
model = VGG()
model = model.to(device)

Pretrained 모델 불러오기

In [49]:
from torchinfo import summary
print('==> Building pre-trained model..')
pre_train_model = origin_dir + '/original_model/model_epoch_120.pth'

# Load checkpoint.
print('==> Resuming from checkpoint..')
checkpoint = torch.load(pre_train_model, map_location=device)
model.load_state_dict(checkpoint)


criterion = nn.CrossEntropyLoss()
feature_result = torch.tensor(0.)
total = torch.tensor(0.)
summary(model,
        input_size=(1,3,32,32),
        col_names=("input_size","output_size","num_params"))

==> Building pre-trained model..
==> Resuming from checkpoint..


Layer (type:depth-idx)                   Input Shape               Output Shape              Param #
VGG                                      [1, 3, 32, 32]            [1, 100]                  --
├─Sequential: 1-1                        [1, 3, 32, 32]            [1, 512, 2, 2]            --
│    └─Conv2d: 2-1                       [1, 3, 32, 32]            [1, 64, 32, 32]           1,792
│    └─BatchNorm2d: 2-2                  [1, 64, 32, 32]           [1, 64, 32, 32]           128
│    └─ReLU: 2-3                         [1, 64, 32, 32]           [1, 64, 32, 32]           --
│    └─Conv2d: 2-4                       [1, 64, 32, 32]           [1, 64, 32, 32]           36,928
│    └─BatchNorm2d: 2-5                  [1, 64, 32, 32]           [1, 64, 32, 32]           128
│    └─ReLU: 2-6                         [1, 64, 32, 32]           [1, 64, 32, 32]           --
│    └─MaxPool2d: 2-7                    [1, 64, 32, 32]           [1, 64, 16, 16]           --
│    └─Conv2d: 2-8        

In [50]:
from torchmetrics import Accuracy
def test_step(model:torch.nn.Module,dataloader:torch.utils.data.DataLoader,loss_fn:torch.nn.Module,accuracy_fn,device:torch.device):
    model.eval()
    test_loss,test_acc=0,0
    testing_loss,testing_acc=0,0
    loss_fn = nn.CrossEntropyLoss(label_smoothing=0.1).to(device)
    accuracy_fn = Accuracy(task="multiclass", num_classes=len(trainset.classes)).to(device)
    with torch.inference_mode():
        test_time_start = timer()
        for image,y in dataloader:
            image,y=image.to(device),y.to(device)
            test_pred=model(image)
            testing_loss+=loss_fn(test_pred,y).item()
            testing_acc+=accuracy_fn(test_pred.argmax(dim=1), y)

        test_loss = testing_loss / len(dataloader)
        test_acc= 100 * testing_acc / len(dataloader)

        test_time_end=timer()
        print(f"\033[94mTest loss is-->{test_loss:.4f}%\nTest acc is-->{test_acc:.4f}%\nDevice is on {str(next(model.parameters()).device)}\nTime passed for testing:{test_time_end-test_time_start:.2f}s\033[0m")

        return test_loss,test_acc


In [51]:
import torch.nn as nn

def evaluate_model(model, dataloader, device="cuda", compress_rate=None):
    """
    Evaluate model and print Top-1 / Top-5 accuracy with neat plain formatting.
    Optionally shows compress_rate.
    """
    model.eval()
    criterion = nn.CrossEntropyLoss()
    top1_correct, top5_correct, total, total_loss = 0, 0, 0, 0.0

    with torch.no_grad():
        for inputs, targets in dataloader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            total_loss += loss.item() * inputs.size(0)
            total += targets.size(0)

            # Top-1 정확도
            _, pred_top1 = outputs.max(1)
            top1_correct += pred_top1.eq(targets).sum().item()

            # Top-5 정확도
            _, pred_top5 = outputs.topk(5, 1, True, True)
            top5_correct += pred_top5.eq(targets.view(-1, 1)).sum().item()

    top1_acc = 100. * top1_correct / total
    top5_acc = 100. * top5_correct / total
    avg_loss = total_loss / total

    # 출력
    print("\n=== Evaluation Results ===")
    if compress_rate is not None:
        print(f" Compress Rate : {compress_rate:.2f}")
    print(f" Loss          : {avg_loss:.4f}")
    print(f" Top-1 Accuracy: {top1_acc:.2f}%")
    print(f" Top-5 Accuracy: {top5_acc:.2f}%")
    print("===========================\n")

    return top1_acc, top5_acc, avg_loss


In [52]:
# 원 모델 평가
evaluate_model(model, testloader, device="cuda")


=== Evaluation Results ===
 Loss          : 1.0332
 Top-1 Accuracy: 75.15%
 Top-5 Accuracy: 92.61%



(75.15, 92.61, 1.0331611846923827)

pruning class 생성

In [53]:
class pruning_vgg_16:
    def __init__(self, model=None, compress_rate=0.5,device=None,
                 origin_dir = '', layer_index=[], out_channel=[],pruning_start_index = 0):
        self.model = model
        self.compress_rate = compress_rate  #각 레이어별 compress rate
        self.device = device
        self.layer_index = layer_index            # 각 conv가 있는 index
        self.out_channel = out_channel
        self.pruning_index = pruning_start_index  # 프루닝할 conv index

    def layer_prune(self, model, conv_num, resume=None, arch="vgg_16_bn"):

        prefix = origin_dir +'/rank_generation/rank_conv'
        subfix = ".npy"                         #.npy(rank generation)이 생긴 폴더로 경로 지정

        for i in range(conv_num):
          conv_i = getattr(model.features, f"conv{self.layer_index[self.pruning_index]}")
          norm_i = getattr(model.features, f"norm{self.layer_index[self.pruning_index]}")
          rank = np.load(prefix + str(self.pruning_index+1) + subfix)       #특정 conv layer rank 불러오기
          pruned_num = int(self.compress_rate * conv_i.out_channels)    #제거할 필터 개수
          ind = np.argsort(rank)[pruned_num:]  # preserved filter id  #오름차순 정렬/남길 index 목록
          keep = [i for i in range(conv_i.out_channels) if i in ind]   #남길 index까지는 생성

          nc = nn.Conv2d(conv_i.in_channels, len(keep), 3, 1, 1)    #첫번째 이후는 계속 바꿔주고 끝나기에 상관없음
          nc.weight.data = conv_i.weight.data[keep].clone()
          if conv_i.bias is not None: nc.bias.data = conv_i.bias.data[keep].clone()
          bn = nn.BatchNorm2d(len(keep))
          bn.weight.data = norm_i.weight.data[keep].clone()
          bn.bias.data = norm_i.bias.data[keep].clone()
          bn.running_mean.data = norm_i.running_mean[keep].clone()
          bn.running_var = norm_i.running_var[keep].clone()
          setattr(model.features, f"conv{self.layer_index[self.pruning_index]}", nc.to(self.device))
          setattr(model.features, f"norm{self.layer_index[self.pruning_index]}", bn.to(self.device))
          self.pruning_index += 1
          if (self.pruning_index == len(self.layer_index)):
            self.transplant_fc_keep_only(model, keep_idx=keep)
          else:  #fc가 아닌경우 / 다음 conv 입력 prune
            old = getattr(model.features, f"conv{self.layer_index[self.pruning_index]}")
            nc_in = nn.Conv2d(len(keep), old.out_channels, 3,1,1)
            nc_in.weight.data = old.weight.data[:, keep].clone()
            setattr(model.features, f"conv{self.layer_index[self.pruning_index]}", nc_in.to(self.device))
            print(f"Pruning conv{layer_index[self.pruning_index-1]} is done")

        return model

    def transplant_fc_keep_only(self, model,
                                sample_loader=None,
                                keep_idx=None,
                                classifier_attr = "classifier"):
        device = next(model.parameters()).device
        model.eval(); model.eval()

        C_full   = self.out_channel[self.pruning_index]
        C_keep   = len(keep_idx)

        cls = model.classifier
        first_i = None

        for i, m in enumerate(cls):
            if isinstance(m, nn.Linear):
                first_i = i; break
        assert first_i is not None, "classifier에 Linear가 없습니다."


        fc: nn.Linear = cls[first_i]

        if sample_loader is not None:
            xb, _ = next(iter(sample_loader))
            xb = xb[:1].to(device)
        else:
            xb = torch.randn(1, 3, 32, 32, device=device)
        with torch.no_grad():
            feat = model.features(xb)
            '''if hasattr(model, "avgpool") and isinstance(model.avgpool, nn.Module):
                feat = model.avgpool(feat)'''
            feat = nn.AvgPool2d(2)(feat)
            _, Ck, H, W = feat.shape
        assert Ck == C_keep, f"추정 C_keep({Ck}) != plan({C_keep})"

        W_old = fc.weight.data.clone()               # (O, C_full*H*W)
        O, in_dim = W_old.shape
        assert in_dim == C_full * H * W, f"원본 FC in_features({in_dim}) != {C_full*H*W}"

        W_old_4d = W_old.view(O, C_full, H, W)         # (O, C_full, H, W) #채널 축 기준 슬라이싱을 위해 4d 정령
        W_new_4d = W_old_4d[:, keep_idx, :, :]         # (O, C_keep, H, W)
        new_fc = nn.Linear(C_keep * H * W, fc.out_features, bias=True).to(device)

        with torch.no_grad():
          new_fc.weight.copy_(W_new_4d.contiguous().view(O,C_keep * H * W))
          if fc.bias is not None:
            new_fc.bias.copy_(fc.bias.data)

        cls[first_i] = new_fc
        setattr(model, classifier_attr, cls)
        print(f"[FC Transplant] in_features: {in_dim} -> {new_fc.in_features}  (kept {len(keep_idx)}/{C_full})")
         ### 사용 예시  transplant_fc_keep_only(pruned, model_orig, plan, sample_loader=calib_loader)

In [39]:
layer_index = [0,1,3,4,6,7,8,10,11,12,14,15,16]
out_channel = [64, 64, 128, 128, 256, 256, 256, 512, 512, 512, 512, 512, 512, 512]
device = next(model.parameters()).device
device

device(type='cuda', index=0)

In [54]:
compress_rate = 0.7
pruning = pruning_vgg_16(model, compress_rate, device, origin_dir, layer_index, out_channel)
arch = 'VGG16'

작업 폴더 생성

In [55]:
saving_folder = job_dir + f'/{compress_rate}_pruned_checkpoint'
if not os.path.isdir(saving_folder):
    os.makedirs(saving_folder)
    print("저장 폴더 생성")
else:
    print("저장 폴더 존재")

저장 폴더 존재


cov_id, resume=None, param_per_cov=4,  arch="vgg_16_bn", conv_num=None):

In [56]:
# Training
def train(epoch, cov_id, optimizer, scheduler, pruning=True):
    print_logger.info('\nEpoch: %d' % epoch)
    model.train()

    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        with torch.cuda.device(device):
            inputs = inputs.to(device)
            targets = targets.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()

            optimizer.step()

            train_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            progress_bar(batch_idx,len(trainloader),
                         'Cov: %d | Loss: %.3f | Acc: %.3f%% (%d/%d)'
                         % (cov_id, train_loss / (batch_idx + 1), 100. * correct / total, correct, total))

def test(epoch, cov_id, optimizer, scheduler):
    top1 = AverageMeter()
    top5 = AverageMeter()

    global best_acc
    model.eval()
    num_iterations = len(testloader)
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            prec1, prec5 = accuracy(outputs, targets, topk=(1, 5))
            top1.update(prec1[0], inputs.size(0))
            top5.update(prec5[0], inputs.size(0))

        print_logger.info(
            'Epoch[{0}]({1}/{2}): '
            'Prec@1(1,5) {top1.avg:.2f}, {top5.avg:.2f}'.format(
                epoch, batch_idx, num_iterations, top1=top1, top5=top5))


In [57]:
print(pruning.pruning_index)
print((len(pruning.layer_index)))
print(compress_rate)

0
13
0.7


Fine-tuning 준비

In [58]:
import torch.optim as optim
from torch.optim.lr_scheduler import LinearLR, ConstantLR, SequentialLR, StepLR

lr = 0.001
optimizer = optim.SGD(model.parameters(), lr=lr,
                          momentum=0.9, weight_decay=5e-4)
warmup_epochs = 5
epochs = 15
scheduler = SequentialLR(
      optimizer,
      schedulers=[
          LinearLR(optimizer, start_factor=lr, total_iters=warmup_epochs),   # ❷
          ConstantLR(optimizer, factor=1.0, total_iters=epochs - warmup_epochs)  # ❸
      ],
      milestones=[warmup_epochs]
  )

In [32]:
layer_conv_num = [2,2,3,3,3]

pruning.pruning_index_index = 0
epochs = 15

for i in range(len(layer_conv_num)):
  model = pruning.layer_prune(conv_num = layer_conv_num[i], model=model)
  print(model)
  optimizer = optim.SGD(model.parameters(), lr=lr,
                          momentum=0.9, weight_decay=5e-4)
  if (i == len(layer_conv_num)-1):
    epochs = 50
    scheduler = SequentialLR(
      optimizer,
      schedulers=[
          LinearLR(optimizer, start_factor=0.0001, total_iters=warmup_epochs),   # ❷
          StepLR(optimizer, step_size=20, gamma=0.1)  # ❸
      ],
      milestones=[warmup_epochs]
  )
  else:
    scheduler = SequentialLR(
      optimizer,
      schedulers=[
          LinearLR(optimizer, start_factor=0.001, total_iters=warmup_epochs),   # ❷
          ConstantLR(optimizer, factor=1.0, total_iters=epochs - warmup_epochs)  # ❸
      ],
      milestones=[warmup_epochs]
  )
  for k in range(epochs):
    train(k+1, i+1, optimizer, scheduler)
    scheduler.step()
    test(k+1, i+1, optimizer, scheduler)
    if(k % 20 == 0):
      torch.save(model.state_dict(), saving_folder + f'/last_model_{k+1}epoches.pth')
      print(f"Saved checkpoint as {job_dir}")
  pruning.pruning_index_index += 1

  torch.save(model.state_dict(), saving_folder + f"/model_{i+1}layer.pth")
  print(f"Saved checkpoint as {job_dir} and Job is completed!")

Pruning conv0 is done


10/03 05:23:06 AM | 
Epoch: 1
INFO:kd:
Epoch: 1


Pruning conv1 is done
VGG(
  (features): Sequential(
    (conv0): Conv2d(3, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (norm0): BatchNorm2d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (conv1): Conv2d(20, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (norm1): BatchNorm2d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu1): ReLU(inplace=True)
    (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv3): Conv2d(20, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (norm3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu3): ReLU(inplace=True)
    (conv4): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (norm4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu4): ReLU(inplace=True)
    (pool5): MaxPool2d(kernel_size=2, s

10/03 05:23:50 AM | Epoch[1](78/79): Prec@1(1,5) 50.23, 74.10
INFO:kd:Epoch[1](78/79): Prec@1(1,5) 50.23, 74.10
10/03 05:23:50 AM | 
Epoch: 2
INFO:kd:
Epoch: 2


Saved checkpoint as /content/drive/MyDrive/Colab_Notebooks/URP/VGG16_CIFAR100_pruning/pruning_job
 [================================================================>]  Step: 36ms | Tot: 39s935ms | Cov: 1 | Loss: 1.246 | Acc: 70.080% (35040/50000) 391/391 


...

 [================================================================>]  Step: 31ms | Tot: 35s982ms | Cov: 5 | Loss: 1.046 | Acc: 71.338% (35669/50000) 391/391 


10/03 06:35:28 AM | Epoch[49](78/79): Prec@1(1,5) 68.23, 90.29
INFO:kd:Epoch[49](78/79): Prec@1(1,5) 68.23, 90.29
10/03 06:35:28 AM | 
Epoch: 50
INFO:kd:
Epoch: 50


 [================================================================>]  Step: 36ms | Tot: 37s482ms | Cov: 5 | Loss: 1.026 | Acc: 71.660% (35830/50000) 391/391 


10/03 06:36:08 AM | Epoch[50](78/79): Prec@1(1,5) 68.07, 90.33
INFO:kd:Epoch[50](78/79): Prec@1(1,5) 68.07, 90.33


Saved checkpoint as /content/drive/MyDrive/Colab_Notebooks/URP/VGG16_CIFAR100_pruning/pruning_job


In [45]:
# 압축률까지 출력
evaluate_model(model, testloader, device="cuda", compress_rate=compress_rate)


=== Evaluation Results ===
 Compress Rate : 0.70
 Loss          : 1.1850
 Top-1 Accuracy: 68.07%
 Top-5 Accuracy: 90.33%



(68.07, 90.33, 1.1849832893371581)

In [46]:
summary(model,
        input_size=(1,3,32,32),
        col_names=("input_size","output_size","num_params"))

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #
VGG                                      [1, 3, 32, 32]            [1, 100]                  --
├─Sequential: 1-1                        [1, 3, 32, 32]            [1, 154, 2, 2]            --
│    └─Conv2d: 2-1                       [1, 3, 32, 32]            [1, 20, 32, 32]           560
│    └─BatchNorm2d: 2-2                  [1, 20, 32, 32]           [1, 20, 32, 32]           40
│    └─ReLU: 2-3                         [1, 20, 32, 32]           [1, 20, 32, 32]           --
│    └─Conv2d: 2-4                       [1, 20, 32, 32]           [1, 20, 32, 32]           3,620
│    └─BatchNorm2d: 2-5                  [1, 20, 32, 32]           [1, 20, 32, 32]           40
│    └─ReLU: 2-6                         [1, 20, 32, 32]           [1, 20, 32, 32]           --
│    └─MaxPool2d: 2-7                    [1, 20, 32, 32]           [1, 20, 16, 16]           --
│    └─Conv2d: 2-8             